In [11]:
import queue 

实验要求：
	编码实现红黑树的插入算法，使得插入后依旧保持红黑性质。（即：实现教材p178页的RB-INSERT,  RB_INSERT_FIXUP算法）
	节点属性：(也可自行定义, 不做强制要求)
		TNode = ｛
					Color：	red / black，
					Key：	int，
					Left：	TNode*， 
					Right：	TNode*，
					P：		TNode*
		｝
程序输入：
		文件名： insert.txt
		文件格式：第一行为待插入数据的个数，第二行为待插入的数据（int类型， 空格分割）
		注：1）初始时红黑树应为空。
			2）按顺序插入， 例如，对于下图的数据，插入顺序应为 20，10，14

程序输出： 
print输出：RB_INSERT_FIXUP算法历经的情况种类数，即代码注释中 case x 的数字，当 z.p==z.p.p.right时（即代码中被省略的else部分），情况种类数顺延为4,5,6。例：书上13-4图中情况种类数输出应为“1 2 3”。
文件输出：将插入完成后的红黑树进行 “层次遍历（Level-Order Traverse）” 并将相应的遍历序列输出到文件中。
		文件名：  LOT.txt     层次遍历序列结果
		格式：每一行对应一个节点的信息（key， color）, nil 节点输出nil即可.
 
实验报告要求：
	1）实验内容
	2）实验目的
	3）算法设计思路
	4）源码 + 注释
	5）算法正确性测试
	6）实验过程中遇到的困难及收获

提交方式：
	提交内容：
		1、源码
		2、实验报告（压缩包命名方式为： 学号-姓名-第x次试验.zip
										例如： SA21011001-张三-第三次试验.zip）		   （2021年12月1日  23：59：59 前）


In [12]:
def load_data():
    file=open("D:\Mis\project\python\dataMine\data\insert.txt")    
    p = file.readlines()
    p[1] = p[1].split(" ")
    l=[]
    for i in p[1]:
        l.append(int(i))
    file.close()
    return l

In [13]:
def out_put_data(res):
    f1 = open('D:\Mis\project\python\dataMine\data\LOT.txt', 'w')
    for i in res:
        if i != 'NIL':
            f1.writelines(['KEY:',str(i[0]),' COLOR:',str(i[1]),'\n'])
        else:
            f1.writelines(['NIL\n'])
    f1.close()

In [14]:
#定义红黑树
class RBTree(object):
    def __init__(self):
        self.nil = RBTreeNode(0)
        self.root = self.nil
class RBTreeNode(object):
    def __init__(self, x):
        self.key = x
        self.left = None
        self.right = None
        self.p = None
        self.color = 'black'
        self.size=None
        self.case = 0 # 默认case为0，取值为0-6


In [15]:
#左旋转
def LeftRotate(T, x):
    y = x.right
    x.right = y.left
    if y.left != T.nil:
        y.left.p = x
    y.p = x.p
    if x.p == T.nil:
        T.root = y
    elif x == x.p.left:
        x.p.left = y
    else:
        x.p.right = y
    y.left = x
    x.p = y



In [16]:
#右旋转
def RightRotate(T, x):
    y = x.left
    x.left = y.right
    if y.right != T.nil:
        y.right.p = x
    y.p = x.p
    if x.p == T.nil:
        T.root = y
    elif x == x.p.right:
        x.p.right = y
    else:
        x.p.left = y
    y.right = x
    x.p = y

In [17]:
#红黑树的插入
def RBInsert( T, z):
    y = T.nil
    x = T.root
    while x != T.nil:
        y = x
        if z.key < x.key:
            x = x.left
        else:
            x = x.right
    z.p = y
    if y == T.nil:
        T.root = z
    elif z.key < y.key:
        y.left = z
    else:
        y.right = z
    z.left = T.nil
    z.right = T.nil
    z.color = 'red'
    RBInsertFixup(T, z)
    return z.key, '颜色为', z.color

def RBInsertFixup(T, z):
    while z.p.color == 'red':
        if z.p == z.p.p.left:  #若“z的父节点”是“z的祖父节点的左孩子”
            y = z.p.p.right
            if y.color == 'red':        #case 1 叔为红色
                z.p.color = 'black'
                y.color = 'black'
                z.p.p.color = 'red'
                z = z.p.p
                z.case = 1
            else:
                if z == z.p.right: 
                    z.case = 2     # Case 2条件：叔叔是黑色，且当前节点是右孩子
                    z = z.p             
                    LeftRotate(T, z)  
                else:   
                    z.case = 3 
                z.p.color = 'black'     #Case 3条件：叔叔是黑色，且当前节点是左孩子。(01) 将“父节点”设为“黑色”。
                z.p.p.color = 'red'     
                RightRotate(T,z.p.p)                    
        else:
            y = z.p.p.left
            if y.color == 'red':
                z.p.color = 'black'
                y.color = 'black'
                z.p.p.color = 'red'
                z = z.p.p
                z.case = 4
            else:
                if z == z.p.left:
                    z = z.p
                    RightRotate(T, z)
                    z.case = 5
                else:
                    z.case = 6
                z.p.color = 'black'
                z.p.p.color = 'red'
                LeftRotate(T, z.p.p)
                   
    T.root.color = 'black'
def RBTransplant( T, u, v):
    if u.p == T.nil:
        T.root = v
    elif u == u.p.left:
        u.p.left = v
    else:
        u.p.right = v
    v.p = u.p



In [18]:
def BFS(T,res):
    q = queue.Queue()
    q.put(T.root)
    
    while not q.empty():
        x = q.get()
        if x == T.nil:
            print("NIL")
            res.append("NIL")
        else:
            print('KEY:', x.key,', COLOR',x.color,", CASE:", x.case)
            res.append((x.key,x.color))
        if x.left!=None:
            q.put(x.left)
        
        if x.right!=None:
            q.put(x.right)

In [19]:
nodes = load_data()
T = RBTree()
print(nodes)
res = []
for node in nodes:
    RBInsert(T,RBTreeNode(node))
#print(T.nil)
BFS(T,res)
out_put_data(res)
print(res)

[12, 1, 9, 2, 0, 11, 7, 19, 4, 15, 18, 5, 14, 13, 10, 16, 6, 3, 8, 17]
KEY: 9 , COLOR black , CASE: 1
KEY: 4 , COLOR black , CASE: 4
KEY: 14 , COLOR black , CASE: 4
KEY: 1 , COLOR red , CASE: 4
KEY: 6 , COLOR red , CASE: 6
KEY: 12 , COLOR red , CASE: 4
KEY: 18 , COLOR red , CASE: 5
KEY: 0 , COLOR black , CASE: 0
KEY: 2 , COLOR black , CASE: 0
KEY: 5 , COLOR black , CASE: 0
KEY: 7 , COLOR black , CASE: 5
KEY: 11 , COLOR black , CASE: 0
KEY: 13 , COLOR black , CASE: 3
KEY: 16 , COLOR black , CASE: 0
KEY: 19 , COLOR black , CASE: 0
NIL
NIL
NIL
KEY: 3 , COLOR red , CASE: 0
NIL
NIL
NIL
KEY: 8 , COLOR red , CASE: 0
KEY: 10 , COLOR red , CASE: 0
NIL
NIL
NIL
KEY: 15 , COLOR red , CASE: 0
KEY: 17 , COLOR red , CASE: 6
NIL
NIL
NIL
NIL
NIL
NIL
NIL
NIL
NIL
NIL
NIL
NIL
[(9, 'black'), (4, 'black'), (14, 'black'), (1, 'red'), (6, 'red'), (12, 'red'), (18, 'red'), (0, 'black'), (2, 'black'), (5, 'black'), (7, 'black'), (11, 'black'), (13, 'black'), (16, 'black'), (19, 'black'), 'NIL', 'NIL', 'NIL', (3